### Lab: Building a Single AI Agent using Strands

This notebook creates a single AI agent using Strands that can guide a home buyer through a preapproval for a new home loan application.
The agent handles creating new home loans and checking the status of existing applications using custom tools.

In [1]:
%pip install -q strands-agents

Note: you may need to restart the kernel to use updated packages.


In [1]:
import os
import json
from datetime import datetime, timedelta
from strands import Agent, tool

### Define Tools

In [2]:
@tool
def get_mortgage_app_doc_status(customer_id: str = None):
    """Retrieves the list of required documents for a mortgage application"""
    return [
        {"type": "proof_of_income", "status": "COMPLETED"},
        {"type": "employment_information", "status": "MISSING"},
        {"type": "proof_of_assets", "status": "COMPLETED"},
        {"type": "credit_information", "status": "COMPLETED"}
    ]

@tool
def get_application_details(customer_id: str = None):
    """Retrieves details about a mortgage application"""
    return {
        "customer_id": customer_id or "123456",
        "application_id": "998776",
        "application_date": str(datetime.today() - timedelta(days=35)),
        "application_status": "IN_PROGRESS",
        "application_type": "NEW_MORTGAGE",
        "name": "Mithil"
    }

@tool
def create_customer_id():
    """Creates a new customer ID"""
    return "123456"

@tool
def create_loan_application(customer_id: str, name: str, age: int, annual_income: int, annual_expense: int):
    """Creates a new loan application"""
    print(f"Creating loan application for customer: {customer_id}")
    print(f"Customer name: {name}")
    print(f"Customer age: {age}")
    print(f"Customer annual income: {annual_income}")
    print(f"Customer annual expense: {annual_expense}")
    return f"Loan application created successfully for {name} (ID: {customer_id})"

### Create the Strands Agent

In [3]:
mortgage_application_agent = Agent(
    model="us.anthropic.claude-3-haiku-20240307-v1:0",
    tools=[get_mortgage_app_doc_status, get_application_details, create_customer_id, create_loan_application],
    system_prompt="""You are a mortgage application agent that helps customers with new mortgage applications.
    
    Instructions:
    - Greet customers warmly before answering
    - First ask for their customer ID, if they don't have one, create a new one
    - Ask for name, age, annual income, and annual expense one question at a time
    - Once you have all information, create the loan application
    - Only discuss mortgage applications, not general mortgage topics
    - Never make up information you cannot retrieve from tools"""
)

print("Mortgage Application Agent created successfully!")

Mortgage Application Agent created successfully!


### Test the Agent

In [4]:
test_queries = [
    "Hello, I'd like to apply for a new mortgage",
    "I don't have a customer ID",
    "My name is John Smith",
    "I'm 35 years old",
    "My annual income is $80000",
    "My annual expenses are $45000"
]

print("Testing Mortgage Application Agent")
print("-" * 40)

for i, query in enumerate(test_queries):
    print(f"\nStep {i+1}: {query}")
    response = mortgage_application_agent(query)
    print(f"Agent: {response.message}")

Testing Mortgage Application Agent
----------------------------------------

Step 1: Hello, I'd like to apply for a new mortgage


SSLError: SSL validation failed for https://bedrock-runtime.us-east-1.amazonaws.com/model/us.anthropic.claude-3-haiku-20240307-v1%3A0/converse-stream [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1000)

### Save the Agent

In [ ]:
import pickle

agent_config = {
    'tools': [get_mortgage_app_doc_status, get_application_details, create_customer_id, create_loan_application],
    'system_prompt': mortgage_application_agent.system_prompt,
    'model': "us.anthropic.claude-3-haiku-20240307-v1:0"
}

with open('mortgage_application_agent_strands.pkl', 'wb') as f:
    pickle.dump(agent_config, f)

print("Agent configuration saved successfully!")
%store mortgage_application_agent